# See if the optimizer can function

In [1]:
from nomelt.thermo_estimation.optimizer import MutationSubsetOptimizer, OptimizerArgs
from nomelt.thermo_estimation import ESMFoldDGEstimator, ESMFoldDGArgs
from nomelt.thermo_estimation.rosetta import RosettaMinimizationParameters
import torch
import optuna
import os

import logging
logger = logging.getLogger('root')
logger.setLevel(logging.DEBUG)
logging.info('test')

[2023-08-07 10:35:17,370] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


2023-08-07 10:35:26.529514: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-07 10:35:26.826690: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, a

In [2]:
torch.cuda.is_available()

True

In [3]:
rosetta_params = RosettaMinimizationParameters(update_pdb=True)
estimator_args = ESMFoldDGArgs(rosetta_params=rosetta_params)

### Use the enh1 and concesus homeodomain as a ground truth test case

In [4]:
_1enh = "RPRTAFSSEQLARLKREFNENRYLTERRRQQLSSELGLNEAQIKIWFQNKRAKI"
_concensus = "RRKRTTFTKEQLEELEELFEKNRYPSAEEREELAKKLGLTERQVKVWFQNRRAKEKK"

In [5]:
estimator = ESMFoldDGEstimator(args=estimator_args)

In [ ]:
ground_results = estimator.run(sequences=[_1enh, _concensus], ids=['wt', 'conc'])

In [10]:
ground_results

{'wt': -130.62494306511954, 'conc': -140.3217549930466}

In [11]:
estimator._gpus_used

{None: ESMFold(
   (esm): ESM2(
     (embed_tokens): Embedding(33, 2560, padding_idx=1)
     (layers): ModuleList(
       (0-35): 36 x TransformerLayer(
         (self_attn): MultiheadAttention(
           (k_proj): Linear(in_features=2560, out_features=2560, bias=True)
           (v_proj): Linear(in_features=2560, out_features=2560, bias=True)
           (q_proj): Linear(in_features=2560, out_features=2560, bias=True)
           (out_proj): Linear(in_features=2560, out_features=2560, bias=True)
           (rot_emb): RotaryEmbedding()
         )
         (self_attn_layer_norm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
         (fc1): Linear(in_features=2560, out_features=10240, bias=True)
         (fc2): Linear(in_features=10240, out_features=2560, bias=True)
         (final_layer_norm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
       )
     )
     (contact_head): ContactPredictionHead(
       (regression): Linear(in_features=1440, out_features=1, bias=True)

#### run the optimizer to see if it can find a better one than the concesus. single gpu

In [6]:
args = OptimizerArgs(
    n_trials=10,
    gapopen=-2,
    penalize_end_gaps=True,
    matrix='BLOSUM62',
    sampler = optuna.samplers.NSGAIISampler(
        population_size=5
    )
    

)
optimizer = MutationSubsetOptimizer(
    _1enh, _concensus, 
    estimator=estimator,
    args=args,
)

INFO:nomelt.thermo_estimation.optimizer:Aligned sequences to determined mutation set:
INFO:nomelt.thermo_estimation.optimizer:
R-PRTAFSSEQLARLKRE-FNE-NRYLT-ERRRQQLSSELGLNEAQIKIWFQNKRA--KI
| .||.|..|||..|. | | | |||.. | .|..|...|||.|.|.|.||||.||  |.
RRKRTTFTKEQLEELE-ELF-EKNRYPSAE-EREELAKKLGLTERQVKVWFQNRRAKEKK
  Score=149

INFO:nomelt.thermo_estimation.optimizer:Aligned sequences with mutations marked:
INFO:nomelt.thermo_estimation.optimizer:
R |  - |  P | RT |  A | F |  S |  S | EQL |  A |  R | L |  K |  R | E |  - | F 
= 0    1    | == 2    | = 3    4    | === 5    6    | = 7    8    | = 9    | = 
R |  R |  K | RT |  T | F |  T |  K | EQL |  E |  E | L |  E |  - | E |  L | F 

|  N | E |  - | NRY |  L |  T |  - | E |  R |  R | R |  Q |  Q | L |  S |  S | 
10   | = 11   | === 12   13   14   | = 15   16   | = 17   18   | = 19   20   21
|  - | E |  K | NRY |  P |  S |  A | E |  - |  E | R |  E |  E | L |  A |  K | 

 E | LGL |  N | E |  A | Q |  I | K |  I | WFQN |  K | RA |  - |  - | K |

In [7]:
optimizer.run()

INFO:nomelt.thermo_estimation.esmfold:Running ESMfold on 2 proteins.
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.73s/it]
INFO:pyrosetta.rosetta:Found rosetta database at: /mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/pyrosetta-2023.27+release.e3ce6ea9faf-py3.9-linux-x86_64.egg/pyrosetta/database; using it....


PyRosetta-4 2023 [Rosetta PyRosetta4.Release.python39.ubuntu 2023.27+release.e3ce6ea9faf661ae8fa769511e2a9b8596417e58 2023-07-07T12:00:46] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.


INFO:pyrosetta.rosetta:PyRosetta-4 2023 [Rosetta PyRosetta4.Release.python39.ubuntu 2023.27+release.e3ce6ea9faf661ae8fa769511e2a9b8596417e58 2023-07-07T12:00:46] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
INFO:rosetta:core.init: Checking for fconfig files in pwd and ./rosetta/flags
INFO:rosetta:core.init: Rosetta version: PyRosetta4.Release.python39.ubuntu r353 2023.27+release.e3ce6ea9faf e3ce6ea9faf661ae8fa769511e2a9b8596417e58 http://www.pyrosetta.org 2023-07-07T12:00:46
INFO:rosetta:core.init: command: PyRosetta -ex1 -ex2aro -database /mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/pyrosetta-2023.27+release.e3ce6ea9faf-py3.9-linux-x86_64.egg/pyrosetta/database
INFO:rosetta:basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=-1010793825 seed_offset=0 real_seed=-1010793825
INFO:rosetta:basic.random.init_ran

  0%|          | 0/10 [00:00<?, ?it/s]

DEBUG:nomelt.thermo_estimation.optimizer:Calling estimator with mutation subset: ['mut_5', 'mut_8', 'mut_16', 'mut_31', 'mut_36', 'mut_42', 'mut_44', 'mut_56']
DEBUG:nomelt.thermo_estimation.optimizer:Variant sequence: RPRTTFSKEQLARLKEFNENRYLTERERQQLASELGLTERQIKIWFQNKRAKKI
INFO:nomelt.thermo_estimation.esmfold:Running ESMfold on 1 proteins.

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.03s/it]
INFO:pyrosetta.rosetta:Found rosetta database at: /mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/pyrosetta-2023.27+release.e3ce6ea9faf-py3.9-linux-x86_64.egg/pyrosetta/database; using it....


...

DEBUG:nomelt.thermo_estimation.optimizer:Calling estimator with mutation subset: ['mut_1', 'mut_5', 'mut_15', 'mut_20', 'mut_26', 'mut_33', 'mut_37', 'mut_38', 'mut_44', 'mut_48', 'mut_53']
DEBUG:nomelt.thermo_estimation.optimizer:Variant sequence: RRPRTTFSSEQLARLEREFENRYPTERRREQLSKKLGLNERQIKVWFQNRRAKI
INFO:nomelt.thermo_estimation.esmfold:Running ESMfold on 1 proteins.

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.03s/it]
INFO:pyrosetta.rosetta:Found rosetta database at: /mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/pyrosetta-2023.27+release.e3ce6ea9faf-py3.9-linux-x86_64.egg/pyrosetta/database; using it....


PyRosetta-4 2023 [Rosetta PyRosetta4.Release.python39.ubuntu 2023.27+release.e3ce6ea9faf661ae8fa769511e2a9b8596417e58 2023-07-07T12:00:46] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.


INFO:pyrosetta.rosetta:PyRosetta-4 2023 [Rosetta PyRosetta4.Release.python39.ubuntu 2023.27+release.e3ce6ea9faf661ae8fa769511e2a9b8596417e58 2023-07-07T12:00:46] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
INFO:rosetta:core.init: Checking for fconfig files in pwd and ./rosetta/flags
INFO:rosetta:core.init: Rosetta version: PyRosetta4.Release.python39.ubuntu r353 2023.27+release.e3ce6ea9faf e3ce6ea9faf661ae8fa769511e2a9b8596417e58 http://www.pyrosetta.org 2023-07-07T12:00:46
INFO:rosetta:core.init: command: PyRosetta -ex1 -ex2aro -database /mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/pyrosetta-2023.27+release.e3ce6ea9faf-py3.9-linux-x86_64.egg/pyrosetta/database
INFO:rosetta:basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=-293160999 seed_offset=0 real_seed=-293160999
INFO:rosetta:basic.random.init_rando

[I 2023-08-07 10:37:38,278] Trial 20 finished with value: -107.33947848284005 and parameters: {'mut_1': True, 'mut_12': False, 'mut_13': False, 'mut_15': True, 'mut_16': False, 'mut_18': False, 'mut_2': False, 'mut_20': True, 'mut_22': False, 'mut_26': True, 'mut_27': False, 'mut_28': False, 'mut_30': False, 'mut_31': False, 'mut_33': True, 'mut_34': False, 'mut_36': False, 'mut_37': True, 'mut_38': True, 'mut_42': False, 'mut_44': True, 'mut_46': False, 'mut_48': True, 'mut_5': True, 'mut_53': True, 'mut_56': False, 'mut_57': False, 'mut_59': False, 'mut_7': False, 'mut_8': False}. Best is trial 15 with value: -142.49927687523726.


({'mut_1': False,
  'mut_12': False,
  'mut_13': True,
  'mut_15': False,
  'mut_16': True,
  'mut_18': True,
  'mut_2': True,
  'mut_20': False,
  'mut_22': True,
  'mut_26': False,
  'mut_27': True,
  'mut_28': False,
  'mut_30': False,
  'mut_31': True,
  'mut_33': False,
  'mut_34': True,
  'mut_36': True,
  'mut_37': True,
  'mut_38': True,
  'mut_42': False,
  'mut_44': False,
  'mut_46': True,
  'mut_48': False,
  'mut_5': False,
  'mut_53': True,
  'mut_56': True,
  'mut_57': False,
  'mut_59': True,
  'mut_7': True,
  'mut_8': True},
 -142.49927687523726,
 <optuna.study.study.Study at 0x14b9c2121280>)

In [8]:
optimizer.best_sequence

'RKRTAFTKEQLAELKELFNEKNRYLSERERQELAKKLGLNEAQVKIWFQNRRAKKK'

### multi gpu

In [9]:
args = OptimizerArgs(
    n_trials=25,
    gapopen=-2,
    penalize_end_gaps=True,
    matrix='BLOSUM62',
    sampler = optuna.samplers.NSGAIISampler(
        population_size=5
    )
    

)
optimizer = MutationSubsetOptimizer(
    _1enh, _concensus, 
    estimator=estimator,
    args=args,
)
optimizer.run(n_jobs=3)

INFO:nomelt.thermo_estimation.optimizer:Aligned sequences to determined mutation set:
INFO:nomelt.thermo_estimation.optimizer:
R-PRTAFSSEQLARLKRE-FNE-NRYLT-ERRRQQLSSELGLNEAQIKIWFQNKRA--KI
| .||.|..|||..|. | | | |||.. | .|..|...|||.|.|.|.||||.||  |.
RRKRTTFTKEQLEELE-ELF-EKNRYPSAE-EREELAKKLGLTERQVKVWFQNRRAKEKK
  Score=149

INFO:nomelt.thermo_estimation.optimizer:Aligned sequences with mutations marked:
INFO:nomelt.thermo_estimation.optimizer:
R |  - |  P | RT |  A | F |  S |  S | EQL |  A |  R | L |  K |  R | E |  - | F 
= 0    1    | == 2    | = 3    4    | === 5    6    | = 7    8    | = 9    | = 
R |  R |  K | RT |  T | F |  T |  K | EQL |  E |  E | L |  E |  - | E |  L | F 

|  N | E |  - | NRY |  L |  T |  - | E |  R |  R | R |  Q |  Q | L |  S |  S | 
10   | = 11   | === 12   13   14   | = 15   16   | = 17   18   | = 19   20   21
|  - | E |  K | NRY |  P |  S |  A | E |  - |  E | R |  E |  E | L |  A |  K | 

 E | LGL |  N | E |  A | Q |  I | K |  I | WFQN |  K | RA |  - |  - | K |

PyRosetta-4 2023 [Rosetta PyRosetta4.Release.python39.ubuntu 2023.27+release.e3ce6ea9faf661ae8fa769511e2a9b8596417e58 2023-07-07T12:00:46] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.


INFO:pyrosetta.rosetta:PyRosetta-4 2023 [Rosetta PyRosetta4.Release.python39.ubuntu 2023.27+release.e3ce6ea9faf661ae8fa769511e2a9b8596417e58 2023-07-07T12:00:46] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
INFO:rosetta:core.init: Checking for fconfig files in pwd and ./rosetta/flags
INFO:rosetta:core.init: Rosetta version: PyRosetta4.Release.python39.ubuntu r353 2023.27+release.e3ce6ea9faf e3ce6ea9faf661ae8fa769511e2a9b8596417e58 http://www.pyrosetta.org 2023-07-07T12:00:46
INFO:rosetta:core.init: command: PyRosetta -ex1 -ex2aro -database /mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/pyrosetta-2023.27+release.e3ce6ea9faf-py3.9-linux-x86_64.egg/pyrosetta/database
INFO:rosetta:basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=933436161 seed_offset=0 real_seed=933436161
INFO:rosetta:basic.random.init_random_

  0%|          | 0/25 [00:00<?, ?it/s]

DEBUG:nomelt.thermo_estimation.optimizer:Calling estimator with mutation subset: ['mut_1', 'mut_7', 'mut_22', 'mut_31', 'mut_33', 'mut_34', 'mut_37', 'mut_42', 'mut_44', 'mut_56']
DEBUG:nomelt.thermo_estimation.optimizer:Variant sequence: RRPRTAFTSEQLARLKREFNEKNRYLTEREREELSKELGLTERQIKIWFQNKRAKKI
INFO:nomelt.thermo_estimation.esmfold:Running ESMfold on 1 proteins.

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.19s/it]
INFO:pyrosetta.rosetta:Found rosetta database at: /mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/pyrosetta-2023.27+release.e3ce6ea9faf-py3.9-linux-x86_64.egg/pyrosetta/database; using it....


...

INFO:pyrosetta.rosetta:PyRosetta-4 2023 [Rosetta PyRosetta4.Release.python39.ubuntu 2023.27+release.e3ce6ea9faf661ae8fa769511e2a9b8596417e58 2023-07-07T12:00:46] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
INFO:rosetta:core.init: Checking for fconfig files in pwd and ./rosetta/flags
INFO:rosetta:core.init: Rosetta version: PyRosetta4.Release.python39.ubuntu r353 2023.27+release.e3ce6ea9faf e3ce6ea9faf661ae8fa769511e2a9b8596417e58 http://www.pyrosetta.org 2023-07-07T12:00:46
INFO:rosetta:core.init: command: PyRosetta -ex1 -ex2aro -database /mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/pyrosetta-2023.27+release.e3ce6ea9faf-py3.9-linux-x86_64.egg/pyrosetta/database
INFO:rosetta:basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=1908955187 seed_offset=0 real_seed=1908955187
INFO:rosetta:basic.random.init_rando

[I 2023-08-07 10:45:45,930] Trial 44 finished with value: -115.14292673047208 and parameters: {'mut_1': False, 'mut_12': False, 'mut_13': True, 'mut_15': False, 'mut_16': True, 'mut_18': True, 'mut_2': False, 'mut_20': False, 'mut_22': True, 'mut_26': False, 'mut_27': True, 'mut_28': False, 'mut_30': False, 'mut_31': True, 'mut_33': False, 'mut_34': True, 'mut_36': True, 'mut_37': True, 'mut_38': False, 'mut_42': False, 'mut_44': True, 'mut_46': True, 'mut_48': False, 'mut_5': True, 'mut_53': True, 'mut_56': True, 'mut_57': False, 'mut_59': False, 'mut_7': True, 'mut_8': True}. Best is trial 36 with value: -149.38617697348528.


INFO:rosetta:core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 985 residue types
INFO:rosetta:core.chemical.GlobalResidueTypeSet: Total time to initialize 0.614874 seconds.
INFO:rosetta:core.import_pose.import_pose: File '/mmfs1/gscratch/cheme/usr/evankomp/repos/nomelt/analysis/tmp/mutation_optimizer_run/-2087766106237805058.pdb' automatically determined to be of type PDB
INFO:rosetta:core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue LYS:CtermProteinFull 55
INFO:rosetta:core.scoring.etable: Starting energy table calculation
INFO:rosetta:core.scoring.etable: smooth_etable: changing atr/rep split to bottom of energy well
INFO:rosetta:core.scoring.etable: smooth_etable: spline smoothing lj etables (maxdis = 6)
INFO:rosetta:core.scoring.etable: smooth_etable: spline smoothing solvation etables (max_dis = 6)
INFO:rosetta:core.scoring.etable: Finished calculating energy tables.
INFO:rosetta:basic.io.database: Databas

[I 2023-08-07 10:45:48,674] Trial 45 finished with value: -149.38617697348528 and parameters: {'mut_1': False, 'mut_12': False, 'mut_13': True, 'mut_15': False, 'mut_16': True, 'mut_18': True, 'mut_2': False, 'mut_20': False, 'mut_22': False, 'mut_26': False, 'mut_27': True, 'mut_28': False, 'mut_30': False, 'mut_31': True, 'mut_33': False, 'mut_34': True, 'mut_36': True, 'mut_37': True, 'mut_38': True, 'mut_42': False, 'mut_44': False, 'mut_46': True, 'mut_48': False, 'mut_5': True, 'mut_53': True, 'mut_56': True, 'mut_57': False, 'mut_59': True, 'mut_7': True, 'mut_8': True}. Best is trial 36 with value: -149.38617697348528.


({'mut_1': False,
  'mut_12': False,
  'mut_13': True,
  'mut_15': False,
  'mut_16': True,
  'mut_18': True,
  'mut_2': False,
  'mut_20': False,
  'mut_22': False,
  'mut_26': False,
  'mut_27': True,
  'mut_28': False,
  'mut_30': False,
  'mut_31': True,
  'mut_33': False,
  'mut_34': True,
  'mut_36': True,
  'mut_37': True,
  'mut_38': True,
  'mut_42': False,
  'mut_44': False,
  'mut_46': True,
  'mut_48': False,
  'mut_5': True,
  'mut_53': True,
  'mut_56': True,
  'mut_57': False,
  'mut_59': True,
  'mut_7': True,
  'mut_8': True},
 -149.38617697348528,
 <optuna.study.study.Study at 0x14b9a573e550>)

In [7]:
optimizer.study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_mut_1,params_mut_12,params_mut_13,params_mut_15,params_mut_16,...,params_mut_48,params_mut_5,params_mut_53,params_mut_56,params_mut_57,params_mut_59,params_mut_7,params_mut_8,system_attrs_nsga2:generation,state
0,0,-118.651758,2023-08-01 12:24:07.284748,2023-08-01 12:24:42.501752,0 days 00:00:35.217004,True,False,False,False,False,...,True,True,False,True,True,True,True,False,0,COMPLETE
1,1,-127.014526,2023-08-01 12:24:42.505533,2023-08-01 12:24:47.337485,0 days 00:00:04.831952,False,False,True,False,False,...,False,False,True,False,True,True,True,False,0,COMPLETE
2,2,-53.574626,2023-08-01 12:24:47.341044,2023-08-01 12:24:51.387568,0 days 00:00:04.046524,False,True,False,False,True,...,False,False,False,False,True,True,True,False,0,COMPLETE
3,3,-136.552455,2023-08-01 12:24:51.390991,2023-08-01 12:24:55.899605,0 days 00:00:04.508614,False,True,False,False,True,...,True,True,False,True,False,False,False,True,0,COMPLETE
4,4,-122.367494,2023-08-01 12:24:55.903087,2023-08-01 12:25:00.286195,0 days 00:00:04.383108,True,True,False,True,False,...,False,False,True,False,False,True,False,True,0,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,-154.781266,2023-08-01 12:31:33.286943,2023-08-01 12:31:37.654102,0 days 00:00:04.367159,True,False,False,True,True,...,False,True,False,True,True,False,True,True,9,COMPLETE
96,96,-154.781266,2023-08-01 12:31:37.657381,2023-08-01 12:31:42.021948,0 days 00:00:04.364567,True,False,False,True,True,...,False,True,False,True,True,False,True,True,9,COMPLETE
97,97,-154.781266,2023-08-01 12:31:42.025390,2023-08-01 12:31:46.394417,0 days 00:00:04.369027,True,False,False,True,True,...,False,True,False,True,True,False,True,True,9,COMPLETE
98,98,-154.781266,2023-08-01 12:31:46.399871,2023-08-01 12:31:50.504547,0 days 00:00:04.104676,True,False,False,True,True,...,False,True,False,True,True,False,True,True,9,COMPLETE


In [10]:
optimizer.optimized_sequence

'RRPRTTFTKEQLERLEELFNEKNRYPSAEEREELAKKLGLNEAQIKIWFQNKRAKEKK'

In [11]:
best_file_hash = optimizer.hash_mutation_set(optimizer.best_mutations)

In [12]:
best_file_hash

-1889697353087351511

In [17]:
os.rename(f'./tmp/mutation_optimizer_run/{best_file_hash}.pdb', f'./tmp/mutation_optimizer_run/consensus_optimized.pdb')

In [1]:
from Bio.PDB import *
import nglview as nv
import ipywidgets
parser = PDBParser()

In [18]:
struct = parser.get_structure('opt', f'./tmp/ESMFoldDGEstimator_structures/{best_file_hash}.pdb')

/mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record '# All ' at line 989
  warnings.warn(
/mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record '#BEGIN' at line 990
  warnings.warn(
/mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'label ' at line 991
  warnings.warn(
/mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'weight' at line 992
  warnings.warn(
/mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecogn

In [19]:
nv.show_biopython(struct, gui=True)

NGLWidget()

In [2]:
wt = parser.get_structure('wt', f'./tmp/ESMFoldDGEstimator_structures/wt.pdb')

/mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record '# All ' at line 958
  warnings.warn(
/mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record '#BEGIN' at line 959
  warnings.warn(
/mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'label ' at line 960
  warnings.warn(
/mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'weight' at line 961
  warnings.warn(
/mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecogn

In [3]:
nv.show_biopython(wt, gui=True)

NGLWidget()

In [4]:
conc = parser.get_structure('conc', f'./tmp/ESMFoldDGEstimator_structures/conc.pdb')

/mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record '# All ' at line 1032
  warnings.warn(
/mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record '#BEGIN' at line 1033
  warnings.warn(
/mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'label ' at line 1034
  warnings.warn(
/mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'weight' at line 1035
  warnings.warn(
/mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unre

In [5]:
nv.show_biopython(conc, gui=True)

NGLWidget()

## Try with the translation

In [6]:
with open('../data/translations.tsv', 'r') as f:
    lines = f.readlines()

In [7]:
translation = lines[1].split()[1]

In [27]:
estimator = ESMFoldDGEstimator(sequences=[_1enh, translation], ids=['wt', 'trans'], args=estimator_args)
ground_results = estimator.run()

INFO:nomelt.thermo_estimation.estimators:Running ESMfold on 2 proteins.
100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.51s/it]


core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 985 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 0.666906 seconds.
core.import_pose.import_pose: File './tmp/ESMFoldDGEstimator_structures/wt.pdb' automatically determined to be of type PDB
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue ILE:CtermProteinFull 54
core.scoring.etable: Starting energy table calculation
core.scoring.etable: smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: smooth_etable: spline smoothing solvation etables (max_dis = 6)
core.scoring.etable: Finished calculating energy tables.
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBPoly1D.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBFadeIntervals.csv
basic.io.databa

DEBUG:nomelt.thermo_estimation.rosetta:Updated PDB file ./tmp/ESMFoldDGEstimator_structures/wt.pdb with minimized positions


core.import_pose.import_pose: File './tmp/ESMFoldDGEstimator_structures/trans.pdb' automatically determined to be of type PDB
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue GLY:CtermProteinFull 86


DEBUG:nomelt.thermo_estimation.rosetta:Updated PDB file ./tmp/ESMFoldDGEstimator_structures/trans.pdb with minimized positions


In [8]:
trans_struct = parser.get_structure('trans', f'./tmp/ESMFoldDGEstimator_structures/trans.pdb')

/mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record '# All ' at line 1543
  warnings.warn(
/mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record '#BEGIN' at line 1544
  warnings.warn(
/mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'label ' at line 1545
  warnings.warn(
/mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'weight' at line 1546
  warnings.warn(
/mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unre

In [9]:
nv.show_biopython(trans_struct, gui=True)

NGLWidget()

In [7]:
args = OptimizerArgs(
    n_trials=100,
    gapopen=-2,
    penalize_end_gaps=False,
    gap_compressed_mutations=True,
    matrix='BLOSUM62',
    sampler = optuna.samplers.NSGAIISampler(
        population_size=10
    )
    

)
optimizer = MutationSubsetOptimizer(
    _1enh, translation, 
    estimator=ESMFoldDGEstimator,
    args=args,
    estimator_args=estimator_args
)

INFO:nomelt.thermo_estimation.optimizer:Aligned sequences to determined mutation set:
INFO:nomelt.thermo_estimation.optimizer:
--RPRTAFSSEQLARLKREFNENRYLTERRRQQLSSELGLNEAQIKIWFQNKRAKI------------------------------
  .|||.|..||.|||..|||..|||...||..|...|.|.|.|.||||||.|.|.                              
MKKPRTSFTNEQIARLEKEFNRQRYLSAQRRAELAASLNLTETQVKIWFQNRRYKTKRKQINQLLGLTEQQVKIWFQNRRYKTKRG
  Score=174

INFO:nomelt.thermo_estimation.optimizer:Compressed gaps where applicable. Orignal mutation set size: 56, new size 26
INFO:nomelt.thermo_estimation.optimizer:Aligned sequences with mutations marked:
INFO:nomelt.thermo_estimation.optimizer:
 |  -- |  R | PRT |  A | F |  S |  S | EQ |  L | ARL |  K |  R | EFN |  E |  N 
 0     1    | === 2    | = 3    4    | == 5    | === 6    7    | === 8    9    
 |  MK |  K | PRT |  S | F |  T |  N | EQ |  I | ARL |  E |  K | EFN |  R |  Q 

| RYL |  T |  E |  R | RR |  Q |  Q | L |  S |  S |  E | L |  G | L |  N | E | 
| === 10   11   12   | == 13   14   | =

In [8]:
optimizer.run()

[I 2023-07-27 15:36:56,329] A new study created in memory with name: no-name-0dcef1b8-946e-4dfc-a976-c3a7f20e084a


  0%|          | 0/100 [00:00<?, ?it/s]

DEBUG:nomelt.thermo_estimation.optimizer:Calling estimator with mutation subset: ['mut_0', 'mut_2', 'mut_6', 'mut_9', 'mut_16', 'mut_17', 'mut_26', 'mut_31', 'mut_34', 'mut_35', 'mut_36', 'mut_53', 'mut_55', 'mut_56']
DEBUG:nomelt.thermo_estimation.optimizer:Variant sequence: MKKPRTSFSNEQLARLEKEFNENRYLSERRRAQLAASLGLNEAQIKIWFQNKRYKTKRKQINQLLGLTEQQVKIWFQNRRYKTKRG
INFO:nomelt.thermo_estimation.estimators:Running ESMfold on 1 proteins.

100%|█████████████████████████████████████████████| 1/1 [00:02<00:00,  2.50s/it]


core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 985 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 0.702705 seconds.
core.import_pose.import_pose: File './tmp/ESMFoldDGEstimator_structures/-1643990933302938567.pdb' automatically determined to be of type PDB
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue GLY:CtermProteinFull 86
core.scoring.etable: Starting energy table calculation
core.scoring.etable: smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: smooth_etable: spline smoothing solvation etables (max_dis = 6)
core.scoring.etable: Finished calculating energy tables.
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBPoly1D.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBFadeIntervals.c

DEBUG:nomelt.thermo_estimation.rosetta:Updated PDB file ./tmp/ESMFoldDGEstimator_structures/-1643990933302938567.pdb with minimized positions
DEBUG:nomelt.thermo_estimation.optimizer:Calling estimator with mutation subset: ['mut_0', 'mut_2', 'mut_12', 'mut_16', 'mut_17', 'mut_21', 'mut_22', 'mut_27', 'mut_28', 'mut_35', 'mut_38', 'mut_53', 'mut_55', 'mut_56']
DEBUG:nomelt.thermo_estimation.optimizer:Variant sequence: MKKPRTAFSSEQIARLEKEFNRQRYLTAQRRQQLSAELNLNEAQIKIWFQNKRYKTKRKQINQLLGLTEQQVKIWFQNRRYKTKRG
INFO:nomelt.thermo_estimation.estimators:Running ESMfold on 1 proteins.


...


100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.18s/it]


core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 985 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 0.68281 seconds.
core.import_pose.import_pose: File './tmp/ESMFoldDGEstimator_structures/1645899727866714080.pdb' automatically determined to be of type PDB
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue GLY:CtermProteinFull 86
core.scoring.etable: Starting energy table calculation
core.scoring.etable: smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: smooth_etable: spline smoothing solvation etables (max_dis = 6)
core.scoring.etable: Finished calculating energy tables.
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBPoly1D.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBFadeIntervals.csv

DEBUG:nomelt.thermo_estimation.rosetta:Updated PDB file ./tmp/ESMFoldDGEstimator_structures/1645899727866714080.pdb with minimized positions


[I 2023-07-27 15:45:31,698] Trial 99 finished with value: -202.10521979763718 and parameters: {'mut_0': True, 'mut_2': True, 'mut_6': False, 'mut_8': True, 'mut_9': True, 'mut_12': False, 'mut_16': True, 'mut_17': False, 'mut_21': True, 'mut_22': False, 'mut_26': False, 'mut_27': False, 'mut_28': True, 'mut_31': False, 'mut_32': True, 'mut_34': True, 'mut_35': True, 'mut_36': True, 'mut_38': False, 'mut_40': True, 'mut_42': True, 'mut_44': True, 'mut_51': False, 'mut_53': True, 'mut_55': False, 'mut_56': True}. Best is trial 47 with value: -207.7674090227535.


({'mut_0': True,
  'mut_2': True,
  'mut_6': True,
  'mut_8': True,
  'mut_9': True,
  'mut_12': False,
  'mut_16': True,
  'mut_17': False,
  'mut_21': False,
  'mut_22': False,
  'mut_26': True,
  'mut_27': False,
  'mut_28': True,
  'mut_31': True,
  'mut_32': True,
  'mut_34': False,
  'mut_35': True,
  'mut_36': True,
  'mut_38': False,
  'mut_40': False,
  'mut_42': True,
  'mut_44': True,
  'mut_51': False,
  'mut_53': True,
  'mut_55': False,
  'mut_56': True},
 -207.7674090227535,
 <optuna.study.study.Study at 0x147199374e80>)

In [12]:
optimizer.optimized_sequence

'MKKPRTSFTNEQLARLEREFNENRYLSEQRRAELSASLGLNETQVKIWFQNKRYKIKRKQINQLLGLTEQQVKIWFQNRRYKTKRG'

In [13]:
best_file_hash = optimizer.hash_mutation_set(optimizer.best_mutations)

In [14]:
best_file_hash

-6688902047467748142

In [17]:
struct = parser.get_structure('opt', f'./tmp/ESMFoldDGEstimator_structures/{best_file_hash}.pdb')

/mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record '# All ' at line 1535
  warnings.warn(
/mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record '#BEGIN' at line 1536
  warnings.warn(
/mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'label ' at line 1537
  warnings.warn(
/mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'weight' at line 1538
  warnings.warn(
/mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unre

In [18]:
nv.show_biopython(struct, gui=True)

NGLWidget()

### repeat with cut version of translation

In [7]:
args = OptimizerArgs(
    n_trials=100,
    gapopen=-2,
    penalize_end_gaps=False,
    gap_compressed_mutations=True,
    matrix='BLOSUM62',
    sampler = optuna.samplers.NSGAIISampler(
        population_size=10
    ),
    cut_tails=2
    

)
optimizer = MutationSubsetOptimizer(
    _1enh, translation, 
    estimator=ESMFoldDGEstimator,
    args=args,
    estimator_args=estimator_args
)

INFO:nomelt.thermo_estimation.optimizer:Aligned sequences to determined mutation set:
INFO:nomelt.thermo_estimation.optimizer:
--RPRTAFSSEQLARLKREFNENRYLTERRRQQLSSELGLNEAQIKIWFQNKRAKI------------------------------
  .|||.|..||.|||..|||..|||...||..|...|.|.|.|.||||||.|.|.                              
MKKPRTSFTNEQIARLEKEFNRQRYLSAQRRAELAASLNLTETQVKIWFQNRRYKTKRKQINQLLGLTEQQVKIWFQNRRYKTKRG
  Score=174

INFO:nomelt.thermo_estimation.optimizer:Aligned sequences after cutting tails:

INFO:nomelt.thermo_estimation.optimizer:--RPRTAFSSEQLARLKREFNENRYLTERRRQQLSSELGLNEAQIKIWFQNKRAKI--
  .|||.|..||.|||..|||..|||...||..|...|.|.|.|.||||||.|.|.  
MKKPRTSFTNEQIARLEKEFNRQRYLSAQRRAELAASLNLTETQVKIWFQNRRYKTKR
INFO:nomelt.thermo_estimation.optimizer:Compressed gaps where applicable. Orignal mutation set size: 28, new size 26
INFO:nomelt.thermo_estimation.optimizer:Aligned sequences with mutations marked:
INFO:nomelt.thermo_estimation.optimizer:
 |  -- |  R | PRT |  A | F |  S |  S | EQ |  L | ARL |  K |  R 

In [ ]:
optimizer.run()

[I 2023-07-31 15:42:10,844] A new study created in memory with name: mutation_optimizer_run


  0%|          | 0/100 [00:00<?, ?it/s]

DEBUG:nomelt.thermo_estimation.optimizer:Calling estimator with mutation subset: ['mut_6', 'mut_8', 'mut_21', 'mut_28', 'mut_34', 'mut_36', 'mut_38', 'mut_42']
DEBUG:nomelt.thermo_estimation.optimizer:Variant sequence: RPRTSFTSEQLARLKREFNRNRYLTEQRRQQLASSLNLNETQIKIWFQNKRAKI
INFO:nomelt.thermo_estimation.estimators:Running ESMfold on 1 proteins.

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.19s/it]
INFO:pyrosetta.rosetta:Found rosetta database at: /mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/pyrosetta-2023.27+release.e3ce6ea9faf-py3.9-linux-x86_64.egg/pyrosetta/database; using it....


PyRosetta-4 2023 [Rosetta PyRosetta4.Release.python39.ubuntu 2023.27+release.e3ce6ea9faf661ae8fa769511e2a9b8596417e58 2023-07-07T12:00:46] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.


INFO:pyrosetta.rosetta:PyRosetta-4 2023 [Rosetta PyRosetta4.Release.python39.ubuntu 2023.27+release.e3ce6ea9faf661ae8fa769511e2a9b8596417e58 2023-07-07T12:00:46] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
INFO:rosetta:core.init: Checking for fconfig files in pwd and ./rosetta/flags
INFO:rosetta:core.init: Rosetta version: PyRosetta4.Release.python39.ubuntu r353 2023.27+release.e3ce6ea9faf e3ce6ea9faf661ae8fa769511e2a9b8596417e58 http://www.pyrosetta.org 2023-07-07T12:00:46
INFO:rosetta:core.init: command: PyRosetta -ex1 -ex2aro -database /mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/pyrosetta-2023.27+release.e3ce6ea9faf-py3.9-linux-x86_64.egg/pyrosetta/database
INFO:rosetta:basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=1572274202 seed_offset=0 real_seed=1572274202
INFO:rosetta:basic.random.init_rando

...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.13s/it]
INFO:pyrosetta.rosetta:Found rosetta database at: /mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/pyrosetta-2023.27+release.e3ce6ea9faf-py3.9-linux-x86_64.egg/pyrosetta/database; using it....


PyRosetta-4 2023 [Rosetta PyRosetta4.Release.python39.ubuntu 2023.27+release.e3ce6ea9faf661ae8fa769511e2a9b8596417e58 2023-07-07T12:00:46] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.


INFO:pyrosetta.rosetta:PyRosetta-4 2023 [Rosetta PyRosetta4.Release.python39.ubuntu 2023.27+release.e3ce6ea9faf661ae8fa769511e2a9b8596417e58 2023-07-07T12:00:46] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
INFO:rosetta:core.init: Checking for fconfig files in pwd and ./rosetta/flags
INFO:rosetta:core.init: Rosetta version: PyRosetta4.Release.python39.ubuntu r353 2023.27+release.e3ce6ea9faf e3ce6ea9faf661ae8fa769511e2a9b8596417e58 http://www.pyrosetta.org 2023-07-07T12:00:46
INFO:rosetta:core.init: command: PyRosetta -ex1 -ex2aro -database /mmfs1/gscratch/cheme/usr/evankomp/mamba/mambaforge/envs/nomelt2/lib/python3.9/site-packages/pyrosetta-2023.27+release.e3ce6ea9faf-py3.9-linux-x86_64.egg/pyrosetta/database
INFO:rosetta:basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=670257654 seed_offset=0 real_seed=670257654
INFO:rosetta:basic.random.init_random_

[I 2023-07-31 15:43:52,762] Trial 6 finished with value: -128.88096008537264 and parameters: {'mut_0': False, 'mut_2': False, 'mut_6': True, 'mut_8': False, 'mut_9': True, 'mut_12': False, 'mut_16': True, 'mut_17': True, 'mut_21': True, 'mut_22': True, 'mut_26': False, 'mut_27': True, 'mut_28': True, 'mut_31': False, 'mut_32': True, 'mut_34': False, 'mut_35': False, 'mut_36': True, 'mut_38': True, 'mut_40': True, 'mut_42': False, 'mut_44': True, 'mut_51': True, 'mut_53': True, 'mut_55': False, 'mut_56': True}. Best is trial 5 with value: -139.21237435239811.



  0%|                                                                                                                                                                             | 0/1 [00:00<?, ?it/s]

In [11]:
optimizer.optimized_sequence

'KPRTAFSSEQLARLEKEFNRNRYLSERRRQELAASLNLTEAQVKIWFQNKRYKIKR'

In [12]:
optimizer.structure_seq_trajectory

[(0,
  'MKRPRTSFTSEQLARLEKEFNRQRYLTARRRAQLSSELNLTEAQVKIWFQNKRAKIKR',
  '/mmfs1/gscratch/cheme/usr/evankomp/repos/nomelt/analysis/tmp/mutation_optimizer_run/-7305603416206835138.pdb'),
 (1,
  'MKKPRTAFSNEQIARLKKEFNRNRYLSEQRRQELSSELGLTEAQVKIWFQNRRYKI',
  '/mmfs1/gscratch/cheme/usr/evankomp/repos/nomelt/analysis/tmp/mutation_optimizer_run/-8907532345303521824.pdb'),
 (2,
  'KPRTSFSSEQLARLKKEFNEQRYLSAQRRAELASSLNLTETQIKIWFQNRRYKI',
  '/mmfs1/gscratch/cheme/usr/evankomp/repos/nomelt/analysis/tmp/mutation_optimizer_run/6370904440427634447.pdb'),
 (3,
  'MKKPRTSFTNEQIARLEKEFNRNRYLTAQRRQQLSSSLNLNEAQIKIWFQNKRAKTKR',
  '/mmfs1/gscratch/cheme/usr/evankomp/repos/nomelt/analysis/tmp/mutation_optimizer_run/3474360579718080596.pdb'),
 (4,
  'MKKPRTAFTNEQIARLKREFNRQRYLTARRRQELSSELGLTEAQVKIWFQNRRYKI',
  '/mmfs1/gscratch/cheme/usr/evankomp/repos/nomelt/analysis/tmp/mutation_optimizer_run/-8776918149564386177.pdb'),
 (5,
  'KPRTAFTNEQLARLEKEFNEQRYLSAQRRQQLAAELNLNEAQIKIWFQNRRAKI',
  '/mmfs1/gscratch/cheme/

In [12]:
import json
with open('./tmp/optimizer_traj.json', 'w') as f:
    json.dump(optimizer.structure_seq_trajectory, f)

DEBUG:filelock:Attempting to acquire lock 22373022625744 on /mmfs1/home/evankomp/.triton/autotune/Fp16Matmul_2d_kernel.pickle.lock
DEBUG:filelock:Lock 22373022625744 acquired on /mmfs1/home/evankomp/.triton/autotune/Fp16Matmul_2d_kernel.pickle.lock
DEBUG:filelock:Attempting to release lock 22373022625744 on /mmfs1/home/evankomp/.triton/autotune/Fp16Matmul_2d_kernel.pickle.lock
DEBUG:filelock:Lock 22373022625744 released on /mmfs1/home/evankomp/.triton/autotune/Fp16Matmul_2d_kernel.pickle.lock
DEBUG:filelock:Attempting to acquire lock 22367615600432 on /mmfs1/home/evankomp/.triton/autotune/Fp16Matmul_4d_kernel.pickle.lock
DEBUG:filelock:Lock 22367615600432 acquired on /mmfs1/home/evankomp/.triton/autotune/Fp16Matmul_4d_kernel.pickle.lock
DEBUG:filelock:Attempting to release lock 22367615600432 on /mmfs1/home/evankomp/.triton/autotune/Fp16Matmul_4d_kernel.pickle.lock
DEBUG:filelock:Lock 22367615600432 released on /mmfs1/home/evankomp/.triton/autotune/Fp16Matmul_4d_kernel.pickle.lock


In [11]:
optimizer.study.trials_dataframe().to_csv('./tmp/optimizer_trials.csv')